## Topic Modeling ##

*This lesson draws on blog posts by [Ted Underwood](https://tedunderwood.com/2012/04/07/topic-modeling-made-just-simple-enough/) and [Matthew Jockers](http://www.matthewjockers.net/2011/09/29/the-lda-buffet-is-now-open-or-latent-dirichlet-allocation-for-english-majors/), this [video of a talk by David Mimno](https://vimeo.com/53080123), and [this notebook](https://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html) by Radim Rehurek. 

In [1]:
# import and setup modules we'll be using in this notebook
import logging # for logging status etc
import itertools # helpful library for iterating through things

import numpy as np # this is a powerful python math package that many others are based on
import gensim # our topic modeling library
import os # for file i/o

# configure logging 
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  

# a helpful function that returns the first `n` elements of the stream as plain list.
# we'll use this later
def head(stream, n=10):
    return list(itertools.islice(stream, n))

In [2]:
# import some more modules for processing the corpus
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

If you haven't already, please download and unzip the Colored Conventions Corpus: [link](http://coloredconventions.org/intro-corpus).

### Tokenizing ###

Many NLP tasks require that you first tokenize your corpus. We actually already tokenized something when we chunked our song lyrics by line. 

Here is [another example of tokenizing](https://programminghistorian.org/en/lessons/sentiment-analysis) that uses nltk to tokenize a document by sentence instead. (Scroll down to where it discusses the nltk word_tokenize module). **ATTENTION! This may be helpful to you for your next homework!**

Here, however, we're going to write our own quick tokenizing function that makes use of gensim's [simple_preprocess function](https://radimrehurek.com/gensim/utils.html), which breaks a document into a list of lowercase tokens. The lower-casing is important for topic modeling since we want both uppercase and lowercase versions of the same word to be counted together. 

In [3]:
# here's some nice dense python for you
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

### Further pre-processing our corpus ###

This is the other necessary step before running a topic model. You need to write a function that iterates through your corpus and returns each document in the format (title, tokens). 

**Side note that we've not yet discussed tuples.** Tuples exist in many programming languages, including R, I think. If not, or even if so, just know that tuples are sequences of objects--  just like lists-- but they cannot be changed. In Python, you indicate a tuple with parentheses. 

In any case, we will want a pre-processing function like this:

In [4]:
# A function to yield each doc in CCP Corpus as a `(filename, tokens)` tuple.

def iter_docs(base_dir):
    docCount = 0
    docs = os.listdir(base_dir)

    for doc in docs:
        if not doc.startswith('.'):
            with open(base_dir + doc, "r") as file:
                text = file.read()
                tokens = tokenize(text) 
        
                yield doc, tokens

In [5]:
# set up the stream for later processing 
stream = iter_docs('./2019-09-ccp-corpus-0.3/ccprecords/')

# while we're at it, take a look at what this looks like for the first five docs
for doc, tokens in itertools.islice(stream, 5):
    print(doc, tokens[:10])  # print the doc title and its first ten tokens

1873.NJ-06.03.NEWB.ART.01.txt ['jersey', 'convention', 'received', 'new', 'citizens', 'state', 'convention', 'new', 'jersey', 'meet']
1884.FL-02.05.GAIN.MIN.01.txt ['proceedings', 'state', 'conference', 'colored', 'men', 'florida', 'held', 'gainesville', 'february', 'address']
1855.CT-04-18.HART.ART.01.txt ['colored', 'men', 'convention', 'according', 'adjournment', 'september', 'colored', 'men', 'connecticut', 'assembled']
1868.IA-02.12.DESM.MIN.01.txt ['proceedings', 'iowa', 'state', 'colored', 'convention', 'held', 'city', 'des', 'moines', 'february']
1872.NE-04.13.OMAH.ART.01.txt ['colored', 'convention', 'following', 'omalia', 'nebraska', 'paper', 'nebraska', 'send', 'delegates', 'new']


The next step is to create a Dictionary (not to be confused with a Python dictionary) which maps each word to a numerical ID. 

This mapping step is required because most algorithms, including gensim's implementation of LDA, rely on numerical libraries that work with vectors indexed by integers, not by strings. Also, many need to know the vector/matrix dimensionality in advance.

The mapping can be constructed automatically by giving gensim's Dictionary class a stream of tokenized documents, like so:

In [6]:
# creating the CCP Corpus Dictionary

doc_stream = (tokens for _, tokens in iter_docs('./2019-09-ccp-corpus-0.3/ccprecords/'))
              
id2word_ccp = gensim.corpora.Dictionary(doc_stream) 

print(id2word_ccp)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(23844 unique tokens: ['ability', 'according', 'advance', 'aim', 'alien']...) from 147 documents (total 469668 corpus positions)


Dictionary(23844 unique tokens: ['ability', 'according', 'advance', 'aim', 'alien']...)


The Dictionary (id2word_ccp) now contains all words that appeared in the corpus, along with how many times they appeared. 

gensim provides a handy function for mapping tokens to their ID numbers, viz:

In [7]:
print(id2word_ccp.token2id)

{'ability': 0, 'according': 1, 'advance': 2, 'aim': 3, 'alien': 4, 'altogether': 5, 'ambition': 6, 'ambitious': 7, 'american': 8, 'ancient': 9, 'appreciate': 10, 'argue': 11, 'aside': 12, 'asked': 13, 'assemblies': 14, 'aves': 15, 'axe': 16, 'believe': 17, 'beneath': 18, 'best': 19, 'bit': 20, 'black': 21, 'born': 22, 'brief': 23, 'brunswick': 24, 'burden': 25, 'called': 26, 'change': 27, 'chiefly': 28, 'citizens': 29, 'class': 30, 'clay': 31, 'colored': 32, 'compromise': 33, 'confess': 34, 'conscience': 35, 'contemptible': 36, 'convention': 37, 'conventionising': 38, 'conventions': 39, 'corner': 40, 'country': 41, 'courageous': 42, 'death': 43, 'demagogue': 44, 'depends': 45, 'depth': 46, 'desires': 47, 'desiring': 48, 'despite': 49, 'dictate': 50, 'direction': 51, 'doubt': 52, 'end': 53, 'endangered': 54, 'endless': 55, 'enfranchisement': 56, 'etrurian': 57, 'exception': 58, 'fact': 59, 'fallen': 60, 'fifths': 61, 'finally': 62, 'fit': 63, 'fittest': 64, 'fo': 65, 'follow': 66, 'forb

There aren't many things you need to do in order to tune your topic model, but one important thing do consider is whether you should filter the words. 

gensim also provides functions for this:

In [8]:
# filter out 50 most frequent words
# id2word_ccp.filter_n_most_frequent(50)

# filter out words in only 1 doc, keeping the rest
# note how no_below and no_above take different values
id2word_ccp.filter_extremes(no_below=2, no_above=1.0)

print(id2word_ccp)

INFO : discarding 9830 tokens: [('aves', 1), ('conventionising', 1), ('etrurian', 1), ('inters', 1), ('molded', 1), ('mounding', 1), ('ordains', 1), ('pitied', 1), ('rarae', 1), ('sunt', 1)]...
INFO : keeping 14014 tokens which were in no less than 2 and no more than 147 (=100.0%) documents
INFO : resulting dictionary: Dictionary(14014 unique tokens: ['ability', 'according', 'advance', 'aim', 'alien']...)


Dictionary(14014 unique tokens: ['ability', 'according', 'advance', 'aim', 'alien']...)


Note that by removing the words that only appeared in a single document, we went from 23,844 unique words (or tokens) to 14,014. That's not a huge number for a topic model, and as you'll see, there are probably other methods that would work better for this corpus. We'll explore some of those next class. 

But for now, since a streamed corpus and a dictionary is all we need to create the vectors for our topic model, we can get started. 

In [9]:
# a class we need; this is the same for every topic model you create with gensim

class Corpus(object):
    def __init__(self, dump_file, dictionary, clip_docs=None):
        self.dump_file = dump_file
        self.dictionary = dictionary
        self.clip_docs = clip_docs
    
    def __iter__(self):
        self.titles = []
        for title, tokens in itertools.islice(iter_docs(self.dump_file), self.clip_docs):
            self.titles.append(title)
            yield self.dictionary.doc2bow(tokens)
    
    def __len__(self):
        return self.clip_docs

In [10]:
# create a stream of bag-of-words vectors
ccp_corpus = Corpus('./2019-09-ccp-corpus-0.3/ccprecords/', id2word_ccp)

# print the first vector in the stream to see what it looks like; 
# this is in the format (word_id, count in first doc)

vector = next(iter(ccp_corpus))
print(vector)  

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 3), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 3), (37, 2), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 1), (90, 3), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 1), (97, 1), (98, 2), (99, 4), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 8), (107, 2), (108, 1), (109, 1), (110, 1),

In [13]:
# now we're ready to run our topic model!

%time lda_model = gensim.models.LdaModel(ccp_corpus, num_topics=15, id2word=id2word_ccp, passes=5) 

# note that passes should be higher -- people usually do 50-100 -- 
# but in the interests of time we'll only do 10 


INFO : using symmetric alpha at 0.06666666666666667
INFO : using symmetric eta at 0.06666666666666667
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 15 topics, 5 passes over the supplied corpus of 147 documents, updating model once every 147 documents, evaluating perplexity every 147 documents, iterating 50x with a convergence threshold of 0.001000
INFO : -10.752 per-word bound, 1724.6 perplexity estimate based on a held-out corpus of 147 documents with 456969 words
INFO : PROGRESS: pass 0, at document #147/147
INFO : topic #10 (0.067): 0.011*"colored" + 0.011*"convention" + 0.010*"committee" + 0.008*"mr" + 0.008*"people" + 0.008*"state" + 0.008*"shall" + 0.005*"men" + 0.005*"motion" + 0.004*"resolved"
INFO : topic #12 (0.067): 0.013*"colored" + 0.013*"committee" + 0.011*"convention" + 0.009*"state" + 0.009*"people" + 0.006*"men" + 0.006*"mr" + 0.006*"shall" + 0.005*"president" + 0.005*"states"
INFO : topic #14 (0.067): 0.011*"convention" 

CPU times: user 30.3 s, sys: 512 ms, total: 30.8 s
Wall time: 23 s


In [ ]:
# some additional helpful functions built into LdaModel

# how to store corpus to disk
from gensim.corpora import MmCorpus
MmCorpus.serialize('./ccp.corpus.mm', ccp_corpus)

# how to store dictionary to disk
id2word_ccp.save('./ccp.dictionary')

# how to store model to disk 
lda_model.save('./lda_ccp-15topics_10iters.model')

In [11]:
# load an old model; in this case, a topic model of the ccp with 50 iterations
lda_model = gensim.models.LdaModel.load('./lda_ccp-15topics_50iters.model')

INFO : loading LdaModel object from ./lda_ccp-15topics_50iters.model
INFO : loading expElogbeta from ./lda_ccp-15topics_50iters.model.expElogbeta.npy with mmap=None
INFO : setting ignored attribute id2word to None
INFO : setting ignored attribute state to None
INFO : setting ignored attribute dispatcher to None
INFO : loaded ./lda_ccp-15topics_50iters.model
INFO : loading LdaState object from ./lda_ccp-15topics_50iters.model.state
INFO : loaded ./lda_ccp-15topics_50iters.model.state


In [12]:
# show the top 20 terms in each topic

lda_model.show_topics(15, 20)

[(0,
  '0.019*"mr" + 0.013*"committee" + 0.009*"convention" + 0.007*"people" + 0.006*"government" + 0.006*"state" + 0.006*"slavery" + 0.006*"report" + 0.005*"motion" + 0.005*"resolution" + 0.005*"men" + 0.005*"cuba" + 0.004*"said" + 0.004*"colored" + 0.003*"adopted" + 0.003*"country" + 0.003*"let" + 0.003*"spanish" + 0.003*"right" + 0.003*"citizens"'),
 (1,
  '0.003*"indiana" + 0.002*"mulattoes" + 0.002*"indianapolis" + 0.002*"persons" + 0.002*"negroes" + 0.001*"speakers" + 0.001*"prevents" + 0.001*"expenditure" + 0.001*"embodies" + 0.001*"loaf" + 0.001*"debarring" + 0.001*"concedes" + 0.001*"sustained" + 0.001*"agitating" + 0.001*"preventing" + 0.001*"extremely" + 0.001*"contain" + 0.001*"episcopal" + 0.001*"lots" + 0.001*"ancestors"'),
 (2,
  '0.012*"shall" + 0.010*"president" + 0.009*"convention" + 0.009*"association" + 0.009*"state" + 0.009*"colored" + 0.008*"county" + 0.007*"committee" + 0.007*"people" + 0.007*"resolved" + 0.006*"men" + 0.006*"rights" + 0.004*"following" + 0.004*"

In [18]:
# let's format the words nicely; 
# the formatted=False parameter returns tuples of (word, probability)

topics = lda_model.show_topics(15, 20, formatted=False)

for topic in topics:
    topic_num = topic[0]
    topic_words = ""
    
    topic_pairs = topic[1]
    for pair in topic_pairs:
        topic_words += pair[0] + ", "
    
    print("T" + str(topic_num) + ": " + topic_words)

T0: mr, committee, convention, people, government, state, slavery, report, motion, resolution, men, cuba, said, colored, adopted, country, let, spanish, right, citizens, 
T1: indiana, mulattoes, indianapolis, persons, negroes, speakers, prevents, expenditure, embodies, loaf, debarring, concedes, sustained, agitating, preventing, extremely, contain, episcopal, lots, ancestors, 
T2: shall, president, convention, association, state, colored, county, committee, people, resolved, men, rights, following, man, motion, slavery, rev, white, citizens, great, 
T3: kentucky, school, state, normal, frankfort, district, wm, louisville, colored, teachers, committee, lexington, laws, weeks, schools, convention, simmons, education, common, ky, 
T4: convention, committee, shall, mr, people, resolved, president, report, motion, appointed, society, rev, new, john, following, states, read, business, subject, seconded, 
T5: colored, state, people, men, convention, states, citizens, white, rights, committee,

In [20]:
# let's take a bit of a closer look at the probabilities attached to each word 
# in a single topic 

# T9 
topic = topics[9]

# this is the topic number
topic_num = topic[0]

topic_pairs = topic[1]
for pair in topic_pairs:
    print(pair[0] + ": " + str(pair[1]))

# since all topics contain all words, the sum of all of the probabilities of each 
# topic should be 1


colored: 0.014209137
people: 0.010636336
white: 0.008736604
south: 0.007967107
conference: 0.0077905497
race: 0.0076308916
negro: 0.007405244
said: 0.0060578035
men: 0.005750052
man: 0.00489279
work: 0.0045141783
great: 0.0044996077
labor: 0.0039110696
time: 0.0038835607
southern: 0.0037885576
country: 0.0037708979
states: 0.0035611608
good: 0.0033553445
let: 0.0032570495
years: 0.0032372389


In [21]:
# let's flip it around and look at the document composition 
# Mallet does this automatically, but with gensim's built-in topic modeling 
# algorithm, we need to do it manually

tokens = [] 

# open one file
with open('./2019-09-ccp-corpus-0.3/ccprecords/1843.NY-08.15.BUFF.MIN.01.txt', "r") as file:
    text = file.read()
    tokens = tokenize(text) # remember this from above

# create the bag of words for the document on the basis of the CCP dictionary, created above
doc_bow = id2word_ccp.doc2bow(tokens)

# get the topics that the doc consists of
doc_topics = lda_model.get_document_topics(doc_bow)

doc_topics
    



[(4, 0.42196515), (14, 0.57217735)]

In [22]:
# now we can cross-reference to find those topics and words

for topic, prob in doc_topics:
    print("T" + str(topic) + ": " + "{:.2%}".format(prob) + " of document.")
          
        #  str(round(prob, 2)))

    topic_words = "Top words in topic: "
    select_topics = topics[topic]
    
    for pair in select_topics[1]:
        topic_words += pair[0] + ", "
    
    print(topic_words)
 

T4: 42.20% of document.
Top words in topic: convention, committee, shall, mr, people, resolved, president, report, motion, appointed, society, rev, new, john, following, states, read, business, subject, seconded, 
T14: 57.22% of document.
Top words in topic: convention, state, people, colored, committee, resolved, shall, resolution, men, man, new, mr, rights, citizens, th, political, states, great, country, free, 


### Evaluating Topics ###

Gensim has several built-in methods for evaluating topics, including something called [topic coherence](https://rare-technologies.com/what-is-topic-coherence/). 

While it's time-consuming, one way to determine whether you've selected the appropriate number of topics is to calculate the coherence score for different numbers of topics. The higher the score, the better.

In [23]:
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=lda_model, corpus=ccp_corpus, coherence='u_mass')

coherence = cm.get_coherence()  # get coherence value

coherence

-0.8975078393034888

But another way to evalute topics is to look at them.

In [ ]:
# LDA visualization tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# just reformat the corpus for pyLDAvis 
ccp_mm_corpus = MmCorpus('./ccp.corpus.mm')

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda_model, ccp_mm_corpus, id2word_ccp)